In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats 
import seaborn as sns
import statsmodels.formula.api as smf
import scikit_posthocs as sp
import seaborn as sns
import itertools
from matplotlib.backends.backend_pdf import PdfPages


sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [2]:
plt.rc("font", size=14)
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import BoundaryNorm, ListedColormap

In [3]:
# columns = ['noun', 'adj', 'propn', 'adv', 'intj', 'cconj', 'sconj', 'adp', 'det', 'num', 'punct', 'symbol', 'pron', 'abbr', 'nominals', 'TTR', 'avg_word_len', 'avr_sent_len', 'hapax_legomena', 'pron/noun_ratio', 'see_pron', 'see_det', '1st_pron', '2nd_pron', '3rd_pron', 'nominalisation', 'active_voice', 'passive_voice', '1st_prs_verb', '2nd_prs_verb', '3rd_prs_verb', 'core_verb', 'verbtype_ratio', 'da_inf', 'inf_verb', 'finite_verb', 'gerund', 'supine', 'verb_particle', 'discourse', 'pres_tense', 'past_tense', 'ind_mood', 'cond_mood', 'imp_mood', 'quot_mood', 'neg_polarity', 'nom_case', 'gen_case', 'part_case', 'ill_case', 'ine_case', 'ela_case', 'alla_case', 'ade_case', 'abl_case', 'tra_case', 'ter_case', 'ess_case', 'abe_case', 'com_case', 'nsubj', 'nsubj_cop', 'modal', 'acl:relc', 'csubj', 'csubj_cop', 'obj', 'ccomp', 'xcomp', 'obl', 'nmod', 'appos', 'nummod', 'amod', 'advcl', 'voc', 'cop', 'conj', 'cc', 'yneemid']
# columns = ['noun', 'adj', 'propn', 'adv', 'intj', 'cconj', 'sconj', 'adp', 'det', 'num', 'punct', 'symbol', 'pron', 'abbr', 'nominals', 'TTR', 'avg_word_len', 'avr_sent_len', 'hapax_legomena', 'pron/noun_ratio', 'see_pron', 'see_det', '1st_pron', '2nd_pron', '3rd_pron', 'nominalisation', 'mine_derivation', 'active_voice', 'passive_voice', '1st_prs_verb', '2nd_prs_verb', '3rd_prs_verb', 'core_verb', 'verbtype_ratio', 'da_inf', 'inf_verb', 'finite_verb', 'gerund', 'supine', 'verb_particle', 'discourse', 'pres_tense', 'past_tense', 'ind_mood', 'cond_mood', 'imp_mood', 'quot_mood', 'neg_polarity', 'nom_case', 'gen_case', 'part_case', 'ill_case', 'ine_case', 'ela_case', 'alla_case', 'ade_case', 'abl_case', 'tra_case', 'ter_case', 'ess_case', 'abe_case', 'com_case', 'nsubj', 'nsubj_cop', 'modal', 'acl:relc', 'csubj', 'csubj_cop', 'obj', 'ccomp', 'xcomp', 'obl', 'nmod', 'appos', 'nummod', 'amod', 'advcl', 'voc', 'cop', 'conj', 'cc', 'yneemid']
columns = ['noun', 'adj', 'propn', 'adv', 'intj', 'cconj', 'sconj', 'adp', 'det', 'num', 'punct', 'symbol', 'pron', 'abbr', 'nominals', 'TTR', 'avg_word_len', 'avr_sent_len', 'hapax_legomena', 'pron/noun_ratio', 'see_pron', 'see_det', '1st_pron_sg', '2nd_pron_sg', '3rd_pron_sg','1st_pron_pl', '2nd_pron_pl', '3rd_pron_pl', 'nominalisation', 'mine_derivation', 'active_voice', 'passive_voice', '1st_prs_verb', '2nd_prs_verb', '3rd_prs_verb', 'core_verb', 'verbtype_ratio', 'da_inf', 'inf_verb', 'finite_verb', 'gerund', 'supine', 'verb_particle', 'discourse', 'pres_tense', 'past_tense', 'ind_mood', 'cond_mood', 'imp_mood', 'quot_mood', 'neg_polarity', 'nom_case', 'gen_case', 'part_case', 'ill_case', 'ine_case', 'ela_case', 'alla_case', 'ade_case', 'abl_case', 'tra_case', 'ter_case', 'ess_case', 'abe_case', 'com_case', 'nsubj', 'nsubj_cop', 'modal', 'acl:relc', 'csubj', 'csubj_cop', 'obj', 'ccomp', 'xcomp', 'obl', 'nmod', 'appos', 'nummod', 'amod', 'advcl', 'voc', 'cop', 'conj', 'cc', 'yneemid']


def get_data(f):
    ls = []
    with open(f, 'r') as fid:
        reader = csv.DictReader(fid, delimiter=';')
        data = [line for line in reader]
        
    return data


### FUNKTSIOONIDE SEKTSIOON

In [4]:
def create_array(data):
    # KW sisendi saamiseks on vaja iga rea (=faili) tunnuste väärtused ehk
    # [[tunnus11, tunnus12,.., tunnus1N], [tunnus21, tunnus22,.., tunnus2N], ..]
    d = [list(line.items()) for line in data]
    arrays = []
    for f in d:
        arrays.append([element[1] for element in f[1:]])
    return np.array(arrays)

In [5]:
def calculate_hb(rank):
    # holm-bonferroni meetodil p väärtuse korrigeerimine, tavalaks on 0.05, aga siin oleme liberaalsed ja panime 0.1
    adjusted_p = 0.1 / (86 - rank + 1) # enne 1 prs pron sg ja pl eraldamist oli tunnuseid 83, seega nende eristamine annab 86
    return adjusted_p

In [6]:
def get_relevant_features(f, p):
    # tagastab listide listi, kus igas listis on [tunnus, järjekorra nr, KW p, korrigeeritud p, saatus]
    features = [[feature, round(p, 5), f] for feature, f, p in zip(columns, f.tolist(), p.tolist())]       
    H_statistics = [feature[2] for feature in features]
    p_values = [feature[1] for feature in features]
    ranks = stats.rankdata(p_values)
    features = [feature[0] for feature in features]
    m = [s for s in zip(features, p_values, H_statistics, ranks)]
    positive_features = [] 
    for el in m:
        kW_p = el[1]
        adjusted_p = calculate_hb(el[3])
        if adjusted_p > kW_p:
            state = 'YES'
        else:
            state = 'NO'
        positive_features.append([el[0], el[3], el[2], kW_p, adjusted_p, state])

    return positive_features  

In [7]:
def calculate_dunn_test2(feature, strong_pd, weak_pd, not_pres_pd):
    data = [strong_pd[feature], weak_pd[feature], notpres_pd[feature]]
    df = pd.DataFrame(data).transpose()

    # gruppide määramine ehk tugev/mõõdukas; nõrk; not_present
    group1 = df.iloc[:, 0].dropna()
    group2 = df.iloc[:, 1].dropna()
    group3 = df.iloc[:, 2].dropna()
    
    dunn = sp.posthoc_dunn([group1, group2, group3], p_adjust='holm')
#         dunn = sp.posthoc_dunn([group1, group2, group3])

    return feature.upper(), dunn


In [8]:
def generate_df(strong_array, weak_array, notpres_array):
    # Kruskal Wallise väljund
    f_statistic, p_value = stats.kruskal(strong_array, weak_array, notpres_array)
    feature_data = get_relevant_features(f_statistic, p_value)

    return feature_data, pd.DataFrame(feature_data, columns =['feature', 'rank', 'H statistic', 'KW p', 'adjusted p', 'state'])

    

In [9]:
def display_posthoc_results(feats, strong, weak, notpres):
    """sisend: tunnused, tugev-mõõdukas, nõrk, puudu dataframe'id.
    siin funktsioonis kasutatakse calculate_dunn_test2 funktsiooni, kus siis tunnus tunnuse haaval vaadatakse konkreetselt ühe tunnuse 
    mõõtmisi kolmes grupis ja arvutatakse, milliste gruppide vahel on erinevusi.
    väljund: list -> tugev-mõõdukas, tugev-puudu, nõrk-puudu grupid per tunnus (kui relevantne tunnuse p on dunni testis <=0.05) 
    """
    
    strong_w = []
    strong_not = []
    weak_not = []
    for feat in feats:
        out = calculate_dunn_test2(feat, strong, weak, notpres)
        print(f'\n{out[0]}\n{out[1]}')
        print('Kui p>0.05, siis need jäetakse välja!\n')
        strong_weak = [out[1].iloc[1, 0], 'tugev/mõõdukas vs nõrk']
        strong_notpres = [out[1].iloc[2, 0], 'tugev/mõõdukas vs puudu']
        weak_notpres = [out[1].iloc[2, 1], 'nõrk vs puudu']
        for score in [strong_weak, strong_notpres, weak_notpres]:
            if score[0] < 0.05:
                if score[1] == 'tugev/mõõdukas vs nõrk':
                    print('GRUPP 1 (tugev/mõõdukas vs nõrk)')
                    print(f'{feat}, {round(score[0], 10)} \n')
                    strong_w.append([feat, round(score[0], 10)])
                    
                if score[1] == 'tugev/mõõdukas vs puudu':
                    print('GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)')
                    print(f'{feat}, {round(score[0], 10)} \n')
                    strong_not.append([feat, round(score[0], 10)])
                if score[1] == 'nõrk vs puudu':
                    print('GRUPP 3 (nõrk vs MITTEEKSISTEERIV)')
                    print(f'{feat}, {round(score[0], 10)} \n')
                    weak_not.append([feat, round(score[0], 10)])
                print(f'TUGEV/MÕÕDUKAS -> mediaan/keskmine: {round(strong[feat].median(), 3)}; {round(strong[feat].mean(), 3)}')
                print(f'NÕRK -> mediaan/keskmine: {round(weak[feat].median(), 3)}; {round(weak[feat].mean(), 3)}')
                print(f'MITTEEKSISTEERIV -> mediaan/keskmine: {round(notpres[feat].median(), 3)}; {round(notpres[feat].mean(), 3)}')


                # print(f'{feat.upper()} =>\n {score[1]}, {round(score[0], 4)}')
            print('================')
    return strong_w, strong_not, weak_not

### INSTRUEERIVUS

In [10]:
f1 = 'dimensioonide_grupid/inst/inst_strong.csv'
f2 = 'dimensioonide_grupid/inst/inst_weak.csv'
f3 = 'dimensioonide_grupid/inst/inst_not_present.csv'

# kruskall Wallise sisendid (kolm gruppi)
strong = get_data(f1)
weak = get_data(f2)
notpres = get_data(f3)


print(f'"tugev/mõõdukas" kokku -> {len(strong)}')
print(f'"nõrk" kokku -> {len(weak)}')
print(f'"puudu" kokku -> {len(notpres)}')


feature_data, df = generate_df(create_array(strong), create_array(weak), create_array(notpres))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)

"tugev/mõõdukas" kokku -> 15
"nõrk" kokku -> 18
"puudu" kokku -> 58


,feature,rank,H statistic,KW p,adjusted p,state
0,noun,24.0,5.997210,0.04986,0.001587,NO
1,adj,65.0,1.372091,0.50356,0.004545,NO
2,propn,11.0,10.179264,0.00616,0.001316,NO
3,adv,31.0,5.319493,0.06997,0.001786,NO
4,intj,81.5,0.199667,0.90499,0.018182,NO
5,cconj,37.0,4.484371,0.10623,0.002000,NO
6,sconj,60.0,1.579174,0.45403,0.003704,NO
7,adp,56.0,1.924492,0.38203,0.003226,NO
8,det,84.0,0.086687,0.95758,0.033333,NO
9,num,46.0,2.714762,0.25733,0.002439,NO


In [11]:
relevant_feats = [feat[0] for feat in feature_data if feat[5] == 'YES']

strong_pd = pd.read_csv(f1, sep=';')
weak_pd = pd.read_csv(f2, sep=';')
notpres_pd = pd.read_csv(f3, sep=';')

for r in relevant_feats:
    print(r)

res1, res2, res3 = display_posthoc_results(relevant_feats, strong_pd, weak_pd, notpres_pd)

pres_tense

PRES_TENSE
          1         2         3
1  1.000000  0.684228  0.001032
2  0.684228  1.000000  0.001824
3  0.001032  0.001824  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
pres_tense, 0.0010315064 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.116; 0.115
NÕRK -> mediaan/keskmine: 0.108; 0.106
MITTEEKSISTEERIV -> mediaan/keskmine: 0.071; 0.071
GRUPP 3 (nõrk vs MITTEEKSISTEERIV)
pres_tense, 0.0018241219 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.116; 0.115
NÕRK -> mediaan/keskmine: 0.108; 0.106
MITTEEKSISTEERIV -> mediaan/keskmine: 0.071; 0.071


### ABSTRAKTSUS

In [12]:
f1 = 'dimensioonide_grupid/abs/abs_strong.csv'
f2 = 'dimensioonide_grupid/abs/abs_weak.csv'
f3 = 'dimensioonide_grupid/abs/abs_not_present.csv'

# kruskall Wallise sisendid (kolm gruppi)
strong = get_data(f1)
weak = get_data(f2)
notpres = get_data(f3)

print(f'"tugev/mõõdukas" kokku -> {len(strong)}')
print(f'"nõrk" kokku -> {len(weak)}')
print(f'"puudu" kokku -> {len(notpres)}')

feature_data, df = generate_df(create_array(strong), create_array(weak), create_array(notpres))

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)
    
relevant_feats = [feat[0] for feat in feature_data if feat[5] == 'YES']

strong_pd = pd.read_csv(f1, sep=';')
weak_pd = pd.read_csv(f2, sep=';')
notpres_pd = pd.read_csv(f3, sep=';')

print(relevant_feats)

res1, res2, res3 = display_posthoc_results(relevant_feats, strong_pd, weak_pd, notpres_pd)

"tugev/mõõdukas" kokku -> 6
"nõrk" kokku -> 36
"puudu" kokku -> 56


,feature,rank,H statistic,KW p,adjusted p,state
0,noun,21.0,6.126720,0.04673,0.001515,NO
1,adj,32.0,3.917161,0.14106,0.001818,NO
2,propn,31.0,4.026985,0.13352,0.001786,NO
3,adv,17.0,6.532639,0.03815,0.001429,NO
4,intj,69.5,0.741086,0.69036,0.005714,NO
5,cconj,62.0,1.166809,0.55800,0.004000,NO
6,sconj,44.0,2.506764,0.28554,0.002326,NO
7,adp,45.0,2.474983,0.29011,0.002381,NO
8,det,16.0,6.563836,0.03756,0.001408,NO
9,num,29.0,4.133290,0.12661,0.001724,NO


['obl']

OBL
          1         2         3
1  1.000000  0.011901  0.000284
2  0.011901  1.000000  0.029788
3  0.000284  0.029788  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 1 (tugev/mõõdukas vs nõrk)
obl, 0.0119008024 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.054; 0.058
NÕRK -> mediaan/keskmine: 0.09; 0.093
MITTEEKSISTEERIV -> mediaan/keskmine: 0.105; 0.106
GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
obl, 0.0002837739 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.054; 0.058
NÕRK -> mediaan/keskmine: 0.09; 0.093
MITTEEKSISTEERIV -> mediaan/keskmine: 0.105; 0.106
GRUPP 3 (nõrk vs MITTEEKSISTEERIV)
obl, 0.0297882814 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.054; 0.058
NÕRK -> mediaan/keskmine: 0.09; 0.093
MITTEEKSISTEERIV -> mediaan/keskmine: 0.105; 0.106


### AFEKTIIVSUS

In [13]:
f1 = 'dimensioonide_grupid/afek/afek_strong.csv'
f2 = 'dimensioonide_grupid/afek/afek_weak.csv'
f3 = 'dimensioonide_grupid/afek/afek_not_present.csv'

# kruskall Wallise sisendid (kolm gruppi)
strong = get_data(f1)
weak = get_data(f2)
notpres = get_data(f3)


print(f'"tugev/mõõdukas" kokku -> {len(strong)}')
print(f'"nõrk" kokku -> {len(weak)}')
print(f'"puudu" kokku -> {len(notpres)}')

feature_data, df = generate_df(create_array(strong), create_array(weak), create_array(notpres))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)


relevant_feats = [feat[0] for feat in feature_data if feat[5] == 'YES']
print(relevant_feats)

strong_pd = pd.read_csv(f1, sep=';')
weak_pd = pd.read_csv(f2, sep=';')
notpres_pd = pd.read_csv(f3, sep=';')


res1, res2, res3 = display_posthoc_results(relevant_feats, strong_pd, weak_pd, notpres_pd)

"tugev/mõõdukas" kokku -> 28
"nõrk" kokku -> 18
"puudu" kokku -> 45


,feature,rank,H statistic,KW p,adjusted p,state
0,noun,9.0,45.329066,0.00000,0.001282,YES
1,adj,32.0,14.991911,0.00056,0.001818,YES
2,propn,27.0,17.490151,0.00016,0.001667,YES
3,adv,9.0,42.370908,0.00000,0.001282,YES
4,intj,24.0,18.676198,0.00009,0.001587,YES
5,cconj,70.0,2.010511,0.36595,0.005882,NO
6,sconj,9.0,29.072826,0.00000,0.001282,YES
7,adp,63.0,2.800776,0.24650,0.004167,NO
8,det,34.0,12.541814,0.00189,0.001887,NO
9,num,49.0,7.097505,0.02876,0.002632,NO


['noun', 'adj', 'propn', 'adv', 'intj', 'sconj', 'pron', 'nominals', 'avg_word_len', 'pron/noun_ratio', 'see_pron', '1st_pron_sg', '2nd_pron_sg', '3rd_pron_sg', '1st_pron_pl', '3rd_pron_pl', 'active_voice', 'passive_voice', '1st_prs_verb', '2nd_prs_verb', 'core_verb', 'finite_verb', 'supine', 'discourse', 'ind_mood', 'imp_mood', 'neg_polarity', 'gen_case', 'ade_case', 'modal', 'xcomp', 'nmod', 'cop']

NOUN
              1         2             3
1  1.000000e+00  0.077622  2.021318e-10
2  7.762157e-02  1.000000  3.966194e-04
3  2.021318e-10  0.000397  1.000000e+00
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
noun, 2e-10 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.22; 0.209
NÕRK -> mediaan/keskmine: 0.241; 0.246
MITTEEKSISTEERIV -> mediaan/keskmine: 0.308; 0.316
GRUPP 3 (nõrk vs MITTEEKSISTEERIV)
noun, 0.0003966194 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.22; 0.209
NÕRK -> mediaan/keskmine: 0.241; 0.246
MITTEEKSISTEERIV -> mediaan/keskmine: 0.308; 0.316


NÕRK -> mediaan/keskmine: 0.0; 0.006
MITTEEKSISTEERIV -> mediaan/keskmine: 0.0; 0.001
GRUPP 3 (nõrk vs MITTEEKSISTEERIV)
1st_pron_pl, 0.0092806156 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.004; 0.005
NÕRK -> mediaan/keskmine: 0.0; 0.006
MITTEEKSISTEERIV -> mediaan/keskmine: 0.0; 0.001

3RD_PRON_PL
          1         2         3
1  1.000000  0.235307  0.000385
2  0.235307  1.000000  0.086820
3  0.000385  0.086820  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
3rd_pron_pl, 0.0003848141 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.001; 0.004
NÕRK -> mediaan/keskmine: 0.0; 0.003
MITTEEKSISTEERIV -> mediaan/keskmine: 0.0; 0.001

ACTIVE_VOICE
          1         2         3
1  1.000000  0.119222  0.000031
2  0.119222  1.000000  0.068387
3  0.000031  0.068387  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
active_voice, 3.11097e-05 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.15; 0.152
NÕRK -> mediaan/keskmine: 


MODAL
              1         2             3
1  1.000000e+00  0.047784  5.079403e-07
2  4.778393e-02  1.000000  3.557906e-02
3  5.079403e-07  0.035579  1.000000e+00
Kui p>0.05, siis need jäetakse välja!

GRUPP 1 (tugev/mõõdukas vs nõrk)
modal, 0.047783929 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.027; 0.028
NÕRK -> mediaan/keskmine: 0.019; 0.02
MITTEEKSISTEERIV -> mediaan/keskmine: 0.007; 0.01
GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
modal, 5.079e-07 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.027; 0.028
NÕRK -> mediaan/keskmine: 0.019; 0.02
MITTEEKSISTEERIV -> mediaan/keskmine: 0.007; 0.01
GRUPP 3 (nõrk vs MITTEEKSISTEERIV)
modal, 0.0355790598 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.027; 0.028
NÕRK -> mediaan/keskmine: 0.019; 0.02
MITTEEKSISTEERIV -> mediaan/keskmine: 0.007; 0.01

XCOMP
          1         2         3
1  1.000000  0.118558  0.000120
2  0.118558  1.000000  0.133277
3  0.000120  0.133277  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEK

### AEG

In [14]:
f1 = 'dimensioonide_grupid/aeg/aeg_strong.csv'
f2 = 'dimensioonide_grupid/aeg/aeg_weak.csv'
f3 = 'dimensioonide_grupid/aeg/aeg_not_present.csv'

# kruskall Wallise sisendid (kolm gruppi)
strong = get_data(f1)
weak = get_data(f2)
notpres = get_data(f3)

print(len(strong))
print(len(weak))
print(len(notpres))

feature_data, df = generate_df(create_array(strong), create_array(weak), create_array(notpres))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)
    
relevant_feats = [feat[0] for feat in feature_data if feat[5] == 'YES']

strong_pd = pd.read_csv(f1, sep=';')
weak_pd = pd.read_csv(f2, sep=';')
notpres_pd = pd.read_csv(f3, sep=';')

print(relevant_feats)

res1, res2, res3 = display_posthoc_results(relevant_feats, strong_pd, weak_pd, notpres_pd)

16
39
19


,feature,rank,H statistic,KW p,adjusted p,state
0,noun,77.0,0.362775,0.83411,0.010000,NO
1,adj,68.0,0.873345,0.64618,0.005263,NO
2,propn,10.0,8.804216,0.01225,0.001299,NO
3,adv,48.0,2.462256,0.29196,0.002564,NO
4,intj,29.5,3.695353,0.15760,0.001739,NO
5,cconj,39.5,2.801630,0.24640,0.002105,NO
6,sconj,17.0,4.806154,0.09044,0.001429,NO
7,adp,73.0,0.814837,0.66537,0.007143,NO
8,det,34.0,3.354441,0.18689,0.001887,NO
9,num,6.0,11.363839,0.00341,0.001235,NO


['da_inf', 'inf_verb', 'pres_tense', 'past_tense', 'nummod']

DA_INF
          1         2         3
1  1.000000  0.011486  0.000284
2  0.011486  1.000000  0.071271
3  0.000284  0.071271  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 1 (tugev/mõõdukas vs nõrk)
da_inf, 0.0114856217 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.009; 0.009
NÕRK -> mediaan/keskmine: 0.017; 0.02
MITTEEKSISTEERIV -> mediaan/keskmine: 0.026; 0.025
GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
da_inf, 0.0002838366 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.009; 0.009
NÕRK -> mediaan/keskmine: 0.017; 0.02
MITTEEKSISTEERIV -> mediaan/keskmine: 0.026; 0.025

INF_VERB
          1         2         3
1  1.000000  0.002599  0.000409
2  0.002599  1.000000  0.224727
3  0.000409  0.224727  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 1 (tugev/mõõdukas vs nõrk)
inf_verb, 0.0025993051 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.041; 0.041
NÕRK -> mediaan/keskmine: 0.058; 0.067
MITTEEKSISTEERIV -> mediaan/keskmine: 0.0

### ARGUMENTATIIVSUS

In [15]:
f1 = 'dimensioonide_grupid/arg/arg_strong.csv'
f2 = 'dimensioonide_grupid/arg/arg_weak.csv'
f3 = 'dimensioonide_grupid/arg/arg_not_present.csv'

# kruskall Wallise sisendid (kolm gruppi)
strong = get_data(f1)
weak = get_data(f2)
notpres = get_data(f3)


# print(len(strong))
# print(len(weak))
# print(len(notpres))

feature_data, df = generate_df(create_array(strong), create_array(weak), create_array(notpres))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)

    
relevant_feats = [feat[0] for feat in feature_data if feat[5] == 'YES']

strong_pd = pd.read_csv(f1, sep=';')
weak_pd = pd.read_csv(f2, sep=';')
notpres_pd = pd.read_csv(f3, sep=';')

print(relevant_feats)

res1, res2, res3 = display_posthoc_results(relevant_feats, strong_pd, weak_pd, notpres_pd)

,feature,rank,H statistic,KW p,adjusted p,state
0,noun,53.0,2.592282,0.27359,0.002941,NO
1,adj,47.0,2.911416,0.23324,0.002500,NO
2,propn,4.0,15.486520,0.00043,0.001205,YES
3,adv,45.0,3.227402,0.19915,0.002381,NO
4,intj,81.5,0.401698,0.81804,0.018182,NO
5,cconj,55.0,2.485287,0.28862,0.003125,NO
6,sconj,23.0,5.617425,0.06028,0.001563,NO
7,adp,68.0,1.037452,0.59528,0.005263,NO
8,det,42.0,3.318292,0.19030,0.002222,NO
9,num,28.0,4.855024,0.08826,0.001695,NO


['propn', 'da_inf', 'pres_tense', 'past_tense', 'neg_polarity', 'modal']

PROPN
          1         2         3
1  1.000000  0.231327  0.000532
2  0.231327  1.000000  0.013760
3  0.000532  0.013760  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
propn, 0.000531673 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.02; 0.032
NÕRK -> mediaan/keskmine: 0.038; 0.047
MITTEEKSISTEERIV -> mediaan/keskmine: 0.076; 0.09
GRUPP 3 (nõrk vs MITTEEKSISTEERIV)
propn, 0.0137603004 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.02; 0.032
NÕRK -> mediaan/keskmine: 0.038; 0.047
MITTEEKSISTEERIV -> mediaan/keskmine: 0.076; 0.09

DA_INF
          1         2         3
1  1.000000  0.999051  0.003872
2  0.999051  1.000000  0.002610
3  0.003872  0.002610  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
da_inf, 0.0038717156 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.025; 0.026
NÕRK -> mediaan/keskmine: 0.021; 0.025
MITTEEKSISTEERIV -> media

### FORMAALSUS

In [16]:
f1 = 'dimensioonide_grupid/form/form_strong.csv'
f2 = 'dimensioonide_grupid/form/form_weak.csv'
f3 = 'dimensioonide_grupid/form/form_not_present.csv'

# kruskall Wallise sisendid (kolm gruppi)
strong = get_data(f1)
weak = get_data(f2)
notpres = get_data(f3)

print(len(strong))
print(len(weak))
print(len(notpres))

feature_data, df = generate_df(create_array(strong), create_array(weak), create_array(notpres))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)
    
relevant_feats = [feat[0] for feat in feature_data if feat[5] == 'YES']
print(relevant_feats)


strong_pd = pd.read_csv(f1, sep=';')
weak_pd = pd.read_csv(f2, sep=';')
notpres_pd = pd.read_csv(f3, sep=';')

res1, res2, res3 = display_posthoc_results(relevant_feats, strong_pd, weak_pd, notpres_pd)

14
28
49


,feature,rank,H statistic,KW p,adjusted p,state
0,noun,6.0,29.979714,0.00000,0.001235,YES
1,adj,25.0,14.089414,0.00087,0.001613,YES
2,propn,14.0,17.691500,0.00014,0.001370,YES
3,adv,6.0,35.727590,0.00000,0.001235,YES
4,intj,51.5,5.057513,0.07976,0.002817,NO
5,cconj,60.0,4.226420,0.12085,0.003704,NO
6,sconj,30.0,12.078198,0.00238,0.001754,NO
7,adp,81.0,0.740396,0.69060,0.016667,NO
8,det,12.5,18.014834,0.00012,0.001342,YES
9,num,45.0,6.669272,0.03563,0.002381,NO


['noun', 'adj', 'propn', 'adv', 'det', 'punct', 'pron', 'nominals', 'avg_word_len', 'avr_sent_len', 'pron/noun_ratio', '1st_pron_sg', '1st_pron_pl', 'active_voice', 'passive_voice', '1st_prs_verb', '2nd_prs_verb', 'core_verb', 'verbtype_ratio', 'finite_verb', 'ind_mood', 'imp_mood', 'gen_case', 'nsubj_cop', 'xcomp', 'nmod', 'cop']

NOUN
          1         2         3
1  1.000000  0.114095  0.000005
2  0.114095  1.000000  0.000173
3  0.000005  0.000173  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
noun, 5.3796e-06 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.335; 0.347
NÕRK -> mediaan/keskmine: 0.299; 0.302
MITTEEKSISTEERIV -> mediaan/keskmine: 0.245; 0.242
GRUPP 3 (nõrk vs MITTEEKSISTEERIV)
noun, 0.0001730261 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.335; 0.347
NÕRK -> mediaan/keskmine: 0.299; 0.302
MITTEEKSISTEERIV -> mediaan/keskmine: 0.245; 0.242

ADJ
          1         2         3
1  1.000000  0.008966  0.000535
2  0.008966  1.000000  0.38


1ST_PRON_PL
          1         2         3
1  1.000000  0.420104  0.004269
2  0.420104  1.000000  0.006021
3  0.004269  0.006021  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
1st_pron_pl, 0.004268646 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.0; 0.0
NÕRK -> mediaan/keskmine: 0.0; 0.002
MITTEEKSISTEERIV -> mediaan/keskmine: 0.0; 0.004
GRUPP 3 (nõrk vs MITTEEKSISTEERIV)
1st_pron_pl, 0.0060214457 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.0; 0.0
NÕRK -> mediaan/keskmine: 0.0; 0.002
MITTEEKSISTEERIV -> mediaan/keskmine: 0.0; 0.004

ACTIVE_VOICE
          1         2         3
1  1.000000  0.543690  0.001672
2  0.543690  1.000000  0.001672
3  0.001672  0.001672  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
active_voice, 0.0016724062 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.111; 0.114
NÕRK -> mediaan/keskmine: 0.116; 0.119
MITTEEKSISTEERIV -> mediaan/keskmine: 0.144; 0.145
GRUPP 3 (nõrk vs MITTEEKSISTE


NMOD
              1         2             3
1  1.000000e+00  0.066031  9.501449e-07
2  6.603068e-02  1.000000  1.261549e-04
3  9.501449e-07  0.000126  1.000000e+00
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
nmod, 9.501e-07 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.15; 0.165
NÕRK -> mediaan/keskmine: 0.11; 0.118
MITTEEKSISTEERIV -> mediaan/keskmine: 0.054; 0.069
GRUPP 3 (nõrk vs MITTEEKSISTEERIV)
nmod, 0.0001261549 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.15; 0.165
NÕRK -> mediaan/keskmine: 0.11; 0.118
MITTEEKSISTEERIV -> mediaan/keskmine: 0.054; 0.069

COP
          1         2         3
1  1.000000  0.012144  0.000225
2  0.012144  1.000000  0.202642
3  0.000225  0.202642  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 1 (tugev/mõõdukas vs nõrk)
cop, 0.0121443064 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.009; 0.011
NÕRK -> mediaan/keskmine: 0.018; 0.021
MITTEEKSISTEERIV -> mediaan/keskmine: 0.022; 0.025
GRUPP 2 (tugev/mõõdukas vs MITTEEKSIST

### IMPERSONAALSUS

In [17]:
f1 = 'dimensioonide_grupid/imp/imp_strong.csv'
f2 = 'dimensioonide_grupid/imp/imp_weak.csv'
f3 = 'dimensioonide_grupid/imp/imp_not_present.csv'

# kruskall Wallise sisendid (kolm gruppi)
strong = get_data(f1)
weak = get_data(f2)
notpres = get_data(f3)


print(len(strong))
print(len(weak))
print(len(notpres))

feature_data, df = generate_df(create_array(strong), create_array(weak), create_array(notpres))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)
    
relevant_feats = [feat[0] for feat in feature_data if feat[5] == 'YES']
print(relevant_feats)

strong_pd = pd.read_csv(f1, sep=';')
weak_pd = pd.read_csv(f2, sep=';')
notpres_pd = pd.read_csv(f3, sep=';')

res1, res2, res3 = display_posthoc_results(relevant_feats, strong_pd, weak_pd, notpres_pd)

37
21
28


,feature,rank,H statistic,KW p,adjusted p,state
0,noun,6.0,41.108480,0.00000,0.001235,YES
1,adj,20.0,15.142822,0.00051,0.001493,YES
2,propn,45.0,5.113597,0.07755,0.002381,NO
3,adv,12.5,23.838630,0.00001,0.001342,YES
4,intj,24.5,13.183002,0.00137,0.001600,YES
5,cconj,68.0,2.258884,0.32321,0.005263,NO
6,sconj,33.0,9.408624,0.00906,0.001852,NO
7,adp,74.0,1.485944,0.47570,0.007692,NO
8,det,22.0,14.350126,0.00077,0.001538,YES
9,num,57.0,4.010251,0.13464,0.003333,NO


['noun', 'adj', 'adv', 'intj', 'det', 'pron', 'abbr', 'nominals', 'avg_word_len', 'pron/noun_ratio', '1st_pron_sg', '1st_pron_pl', '3rd_pron_pl', 'active_voice', 'passive_voice', '1st_prs_verb', '2nd_prs_verb', 'core_verb', 'verbtype_ratio', 'finite_verb', 'supine', 'discourse', 'ind_mood', 'imp_mood', 'gen_case', 'xcomp', 'nmod']

NOUN
              1         2             3
1  1.000000e+00  0.000986  7.810205e-10
2  9.864760e-04  1.000000  2.876591e-02
3  7.810205e-10  0.028766  1.000000e+00
Kui p>0.05, siis need jäetakse välja!

GRUPP 1 (tugev/mõõdukas vs nõrk)
noun, 0.000986476 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.321; 0.328
NÕRK -> mediaan/keskmine: 0.262; 0.263
MITTEEKSISTEERIV -> mediaan/keskmine: 0.216; 0.214
GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
noun, 8e-10 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.321; 0.328
NÕRK -> mediaan/keskmine: 0.262; 0.263
MITTEEKSISTEERIV -> mediaan/keskmine: 0.216; 0.214
GRUPP 3 (nõrk vs MITTEEKSISTEERIV)
noun, 0.0287659082 

TUGEV/MÕÕDUKAS -

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.0; 0.0
NÕRK -> mediaan/keskmine: 0.0; 0.003
MITTEEKSISTEERIV -> mediaan/keskmine: 0.004; 0.012

1ST_PRON_PL
          1         2         3
1  1.000000  0.620232  0.000035
2  0.620232  1.000000  0.001723
3  0.000035  0.001723  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
1st_pron_pl, 3.54785e-05 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.0; 0.001
NÕRK -> mediaan/keskmine: 0.0; 0.001
MITTEEKSISTEERIV -> mediaan/keskmine: 0.004; 0.007
GRUPP 3 (nõrk vs MITTEEKSISTEERIV)
1st_pron_pl, 0.0017233055 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.0; 0.001
NÕRK -> mediaan/keskmine: 0.0; 0.001
MITTEEKSISTEERIV -> mediaan/keskmine: 0.004; 0.007

3RD_PRON_PL
          1         2         3
1  1.000000  0.154277  0.000252
2  0.154277  1.000000  0.154277
3  0.000252  0.154277  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
3rd_pron_pl, 0.000251957 

TUGEV/MÕÕDUKAS -> mediaan/kesk


GEN_CASE
          1         2         3
1  1.000000  0.135766  0.000711
2  0.135766  1.000000  0.143838
3  0.000711  0.143838  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
gen_case, 0.0007107731 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.166; 0.156
NÕRK -> mediaan/keskmine: 0.124; 0.122
MITTEEKSISTEERIV -> mediaan/keskmine: 0.092; 0.1

XCOMP
          1         2         3
1  1.000000  0.007098  0.000002
2  0.007098  1.000000  0.129041
3  0.000002  0.129041  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 1 (tugev/mõõdukas vs nõrk)
xcomp, 0.0070980169 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.006; 0.007
NÕRK -> mediaan/keskmine: 0.013; 0.015
MITTEEKSISTEERIV -> mediaan/keskmine: 0.019; 0.018
GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
xcomp, 2.4714e-06 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.006; 0.007
NÕRK -> mediaan/keskmine: 0.013; 0.015
MITTEEKSISTEERIV -> mediaan/keskmine: 0.019; 0.018

NMOD
              1         2             

### INFOTIHEDUS

In [18]:
f1 = 'dimensioonide_grupid/info/info_strong.csv'
f2 = 'dimensioonide_grupid/info/info_weak.csv'
f3 = 'dimensioonide_grupid/info/info_not_present.csv'

# kruskall Wallise sisendid (kolm gruppi)
strong = get_data(f1)
weak = get_data(f2)
notpres = get_data(f3)

print(len(strong))
print(len(weak))
print(len(notpres))

feature_data, df = generate_df(create_array(strong), create_array(weak), create_array(notpres))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)
    
relevant_feats = [feat[0] for feat in feature_data if feat[5] == 'YES']
print(relevant_feats)

strong_pd = pd.read_csv(f1, sep=';')
weak_pd = pd.read_csv(f2, sep=';')
notpres_pd = pd.read_csv(f3, sep=';')

res1, res2, res3 = display_posthoc_results(relevant_feats, strong_pd, weak_pd, notpres_pd)

45
26
5


,feature,rank,H statistic,KW p,adjusted p,state
0,noun,16.0,14.593942,0.00068,0.001408,YES
1,adj,39.0,6.760937,0.03403,0.002083,NO
2,propn,31.0,8.954491,0.01136,0.001786,NO
3,adv,14.0,15.668664,0.00040,0.001370,YES
4,intj,28.5,9.315008,0.00949,0.001709,NO
5,cconj,68.0,0.852564,0.65293,0.005263,NO
6,sconj,19.0,10.851800,0.00440,0.001471,NO
7,adp,55.0,2.967836,0.22675,0.003125,NO
8,det,20.0,10.765474,0.00460,0.001493,NO
9,num,10.5,19.460002,0.00006,0.001307,YES


['noun', 'adv', 'num', 'pron', 'nominals', 'avg_word_len', 'pron/noun_ratio', '1st_pron_sg', '1st_pron_pl', '1st_prs_verb', '2nd_prs_verb', 'finite_verb', 'imp_mood', 'neg_polarity', 'xcomp', 'nummod']

NOUN
          1         2         3
1  1.000000  0.002513  0.027851
2  0.002513  1.000000  0.490774
3  0.027851  0.490774  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 1 (tugev/mõõdukas vs nõrk)
noun, 0.0025134747 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.302; 0.3
NÕRK -> mediaan/keskmine: 0.236; 0.247
MITTEEKSISTEERIV -> mediaan/keskmine: 0.219; 0.23
GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
noun, 0.0278508216 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.302; 0.3
NÕRK -> mediaan/keskmine: 0.236; 0.247
MITTEEKSISTEERIV -> mediaan/keskmine: 0.219; 0.23

ADV
          1         2         3
1  1.000000  0.000531  0.099573
2  0.000531  1.000000  0.998293
3  0.099573  0.998293  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 1 (tugev/mõõdukas vs nõrk)
adv, 0.0005307072 

TUGEV/MÕ


XCOMP
          1         2         3
1  1.000000  0.000117  0.022626
2  0.000117  1.000000  0.710998
3  0.022626  0.710998  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 1 (tugev/mõõdukas vs nõrk)
xcomp, 0.000117423 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.008; 0.01
NÕRK -> mediaan/keskmine: 0.018; 0.019
MITTEEKSISTEERIV -> mediaan/keskmine: 0.018; 0.021
GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
xcomp, 0.0226264915 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.008; 0.01
NÕRK -> mediaan/keskmine: 0.018; 0.019
MITTEEKSISTEERIV -> mediaan/keskmine: 0.018; 0.021

NUMMOD
          1         2         3
1  1.000000  0.000035  0.647305
2  0.000035  1.000000  0.153838
3  0.647305  0.153838  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 1 (tugev/mõõdukas vs nõrk)
nummod, 3.52914e-05 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.028; 0.033
NÕRK -> mediaan/keskmine: 0.008; 0.009
MITTEEKSISTEERIV -> mediaan/keskmine: 0.019; 0.025


### INTERAKTIIVSUS

In [19]:
f1 = 'dimensioonide_grupid/inter/inter_strong.csv'
f2 = 'dimensioonide_grupid/inter/inter_weak.csv'
f3 = 'dimensioonide_grupid/inter/inter_not_present.csv'

# kruskall Wallise sisendid (kolm gruppi)
strong = get_data(f1)
weak = get_data(f2)
notpres = get_data(f3)

print(len(strong))
print(len(weak))
print(len(notpres))

feature_data, df = generate_df(create_array(strong), create_array(weak), create_array(notpres))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)
    
relevant_feats = [feat[0] for feat in feature_data if feat[5] == 'YES']
print(relevant_feats)

strong_pd = pd.read_csv(f1, sep=';')
weak_pd = pd.read_csv(f2, sep=';')
notpres_pd = pd.read_csv(f3, sep=';')

res1, res2, res3 = display_posthoc_results(relevant_feats, strong_pd, weak_pd, notpres_pd)

10
15
75


,feature,rank,H statistic,KW p,adjusted p,state
0,noun,7.0,23.334374,0.00001,0.001250,YES
1,adj,29.0,10.943733,0.00420,0.001724,NO
2,propn,30.0,10.678939,0.00480,0.001754,NO
3,adv,24.0,12.668687,0.00177,0.001587,NO
4,intj,10.5,20.476598,0.00004,0.001307,YES
5,cconj,85.0,0.073968,0.96369,0.050000,NO
6,sconj,39.0,7.137616,0.02819,0.002083,NO
7,adp,53.5,3.616788,0.16392,0.002985,NO
8,det,52.0,3.726473,0.15517,0.002857,NO
9,num,38.0,7.203906,0.02727,0.002041,NO


['noun', 'intj', 'pron', 'nominals', 'TTR', 'avg_word_len', 'hapax_legomena', 'pron/noun_ratio', '1st_pron_sg', '2nd_pron_sg', '1st_pron_pl', '2nd_pron_pl', '1st_prs_verb', '2nd_prs_verb', 'core_verb', 'finite_verb', 'discourse', 'imp_mood', 'neg_polarity', 'gen_case', 'modal', 'nmod', 'voc']

NOUN
          1         2         3
1  1.000000  0.522908  0.000532
2  0.522908  1.000000  0.000798
3  0.000532  0.000798  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
noun, 0.0005316103 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.212; 0.205
NÕRK -> mediaan/keskmine: 0.227; 0.222
MITTEEKSISTEERIV -> mediaan/keskmine: 0.286; 0.289
GRUPP 3 (nõrk vs MITTEEKSISTEERIV)
noun, 0.0007983047 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.212; 0.205
NÕRK -> mediaan/keskmine: 0.227; 0.222
MITTEEKSISTEERIV -> mediaan/keskmine: 0.286; 0.289

INTJ
          1         2         3
1  1.000000  0.026642  0.000033
2  0.026642  1.000000  0.096095
3  0.000033  0.096095  1.000000


2ND_PRS_VERB
              1         2             3
1  1.000000e+00  0.005915  4.474193e-10
2  5.915001e-03  1.000000  5.201339e-04
3  4.474193e-10  0.000520  1.000000e+00
Kui p>0.05, siis need jäetakse välja!

GRUPP 1 (tugev/mõõdukas vs nõrk)
2nd_prs_verb, 0.0059150013 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.01; 0.015
NÕRK -> mediaan/keskmine: 0.003; 0.007
MITTEEKSISTEERIV -> mediaan/keskmine: 0.0; 0.003
GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
2nd_prs_verb, 4e-10 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.01; 0.015
NÕRK -> mediaan/keskmine: 0.003; 0.007
MITTEEKSISTEERIV -> mediaan/keskmine: 0.0; 0.003
GRUPP 3 (nõrk vs MITTEEKSISTEERIV)
2nd_prs_verb, 0.0005201339 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.01; 0.015
NÕRK -> mediaan/keskmine: 0.003; 0.007
MITTEEKSISTEERIV -> mediaan/keskmine: 0.0; 0.003

CORE_VERB
          1         2         3
1  1.000000  0.859229  0.034803
2  0.859229  1.000000  0.006076
3  0.034803  0.006076  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 

### KEERULISUS

In [20]:
f1 = 'dimensioonide_grupid/keer/keer_strong.csv'
f2 = 'dimensioonide_grupid/keer/keer_weak.csv'
f3 = 'dimensioonide_grupid/keer/keer_not_present.csv'

# kruskall Wallise sisendid (kolm gruppi)
strong = get_data(f1)
weak = get_data(f2)
notpres = get_data(f3)

print(len(strong))
print(len(weak))
print(len(notpres))

feature_data, df = generate_df(create_array(strong), create_array(weak), create_array(notpres))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)
    
relevant_feats = [feat[0] for feat in feature_data if feat[5] == 'YES']
print(relevant_feats)

strong_pd = pd.read_csv(f1, sep=';')
weak_pd = pd.read_csv(f2, sep=';')
notpres_pd = pd.read_csv(f3, sep=';')

res1, res2, res3 = display_posthoc_results(relevant_feats, strong_pd, weak_pd, notpres_pd)

9
25
53


,feature,rank,H statistic,KW p,adjusted p,state
0,noun,18.0,9.083589,0.01065,0.001449,NO
1,adj,25.0,7.830421,0.01994,0.001613,NO
2,propn,28.0,7.123718,0.02839,0.001695,NO
3,adv,30.0,6.994662,0.03028,0.001754,NO
4,intj,44.0,4.079898,0.13004,0.002326,NO
5,cconj,81.0,0.387453,0.82388,0.016667,NO
6,sconj,51.0,3.581159,0.16686,0.002778,NO
7,adp,66.0,1.275010,0.52861,0.004762,NO
8,det,42.0,4.124993,0.12714,0.002222,NO
9,num,52.0,3.533189,0.17091,0.002857,NO


['punct', 'abbr', '1st_prs_verb', 'finite_verb', 'nmod']

PUNCT
          1         2         3
1  1.000000  0.637972  0.016343
2  0.637972  1.000000  0.004476
3  0.016343  0.004476  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
punct, 0.0163434264 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.118; 0.127
NÕRK -> mediaan/keskmine: 0.125; 0.126
MITTEEKSISTEERIV -> mediaan/keskmine: 0.148; 0.148
GRUPP 3 (nõrk vs MITTEEKSISTEERIV)
punct, 0.0044760158 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.118; 0.127
NÕRK -> mediaan/keskmine: 0.125; 0.126
MITTEEKSISTEERIV -> mediaan/keskmine: 0.148; 0.148

ABBR
          1         2         3
1  1.000000  0.054014  0.000676
2  0.054014  1.000000  0.054014
3  0.000676  0.054014  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
abbr, 0.0006755665 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.054; 0.045
NÕRK -> mediaan/keskmine: 0.008; 0.017
MITTEEKSISTEERIV -> mediaan/keskmine: 0.

### SPONTAANSUS

In [21]:
f1 = 'dimensioonide_grupid/spont/spont_strong.csv'
f2 = 'dimensioonide_grupid/spont/spont_weak.csv'
f3 = 'dimensioonide_grupid/spont/spont_not_present.csv'

# kruskall Wallise sisendid (kolm gruppi)
strong = get_data(f1)
weak = get_data(f2)
notpres = get_data(f3)


print(len(strong))
print(len(weak))
print(len(notpres))

feature_data, df = generate_df(create_array(strong), create_array(weak), create_array(notpres))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)
    
relevant_feats = [feat[0] for feat in feature_data if feat[5] == 'YES']
print(relevant_feats)

strong_pd = pd.read_csv(f1, sep=';')
weak_pd = pd.read_csv(f2, sep=';')
notpres_pd = pd.read_csv(f3, sep=';')

res1, res2, res3 = display_posthoc_results(relevant_feats, strong_pd, weak_pd, notpres_pd)

12
6
79


,feature,rank,H statistic,KW p,adjusted p,state
0,noun,4.0,26.278573,0.00000,0.001205,YES
1,adj,41.0,6.994455,0.03028,0.002174,NO
2,propn,22.0,15.042598,0.00054,0.001538,YES
3,adv,9.5,22.927600,0.00001,0.001290,YES
4,intj,4.0,38.165935,0.00000,0.001205,YES
5,cconj,70.0,1.597142,0.44997,0.005882,NO
6,sconj,49.0,4.272349,0.11811,0.002632,NO
7,adp,63.0,2.331758,0.31165,0.004167,NO
8,det,55.0,3.378742,0.18464,0.003125,NO
9,num,43.0,5.243742,0.07267,0.002273,NO


['noun', 'propn', 'adv', 'intj', 'pron', 'nominals', 'TTR', 'avg_word_len', 'pron/noun_ratio', '1st_pron_sg', '2nd_pron_sg', '2nd_pron_pl', '1st_prs_verb', '2nd_prs_verb', 'core_verb', 'finite_verb', 'discourse', 'imp_mood', 'neg_polarity', 'gen_case', 'nsubj_cop', 'obl', 'nmod', 'cop']

NOUN
          1         2         3
1  1.000000  0.473671  0.000009
2  0.473671  1.000000  0.020539
3  0.000009  0.020539  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
noun, 9.2864e-06 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.212; 0.198
NÕRK -> mediaan/keskmine: 0.231; 0.237
MITTEEKSISTEERIV -> mediaan/keskmine: 0.296; 0.296
GRUPP 3 (nõrk vs MITTEEKSISTEERIV)
noun, 0.0205386879 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.212; 0.198
NÕRK -> mediaan/keskmine: 0.231; 0.237
MITTEEKSISTEERIV -> mediaan/keskmine: 0.296; 0.296

PROPN
          1         2         3
1  1.000000  0.618473  0.001343
2  0.618473  1.000000  0.095346
3  0.001343  0.095346  1.000000
Kui p>


CORE_VERB
          1         2         3
1  1.000000  0.868283  0.003013
2  0.868283  1.000000  0.054104
3  0.003013  0.054104  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
core_verb, 0.0030131014 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.036; 0.034
NÕRK -> mediaan/keskmine: 0.031; 0.034
MITTEEKSISTEERIV -> mediaan/keskmine: 0.017; 0.02

FINITE_VERB
          1         2         3
1  1.000000  0.799005  0.000813
2  0.799005  1.000000  0.036192
3  0.000813  0.036192  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
finite_verb, 0.0008129952 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.133; 0.131
NÕRK -> mediaan/keskmine: 0.123; 0.125
MITTEEKSISTEERIV -> mediaan/keskmine: 0.105; 0.105
GRUPP 3 (nõrk vs MITTEEKSISTEERIV)
finite_verb, 0.0361924585 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.133; 0.131
NÕRK -> mediaan/keskmine: 0.123; 0.125
MITTEEKSISTEERIV -> mediaan/keskmine: 0.105; 0.105

DISCOURSE
        

### SUBJEKTIIVSUS

In [22]:
f1 = 'dimensioonide_grupid/subj/subj_strong.csv'
f2 = 'dimensioonide_grupid/subj/subj_weak.csv'
f3 = 'dimensioonide_grupid/subj/subj_not_present.csv'

# kruskall Wallise sisendid (kolm gruppi)
strong = get_data(f1)
weak = get_data(f2)
notpres = get_data(f3)

print(len(strong))
print(len(weak))
print(len(notpres))

feature_data, df = generate_df(create_array(strong), create_array(weak), create_array(notpres))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)
    
relevant_feats = [feat[0] for feat in feature_data if feat[5] == 'YES']
print(relevant_feats)

strong_pd = pd.read_csv(f1, sep=';')
weak_pd = pd.read_csv(f2, sep=';')
notpres_pd = pd.read_csv(f3, sep=';')

res1, res2, res3 = display_posthoc_results(relevant_feats, strong_pd, weak_pd, notpres_pd)

30
15
52


,feature,rank,H statistic,KW p,adjusted p,state
0,noun,8.5,38.407955,0.00000,0.001274,YES
1,adj,28.0,15.569052,0.00042,0.001695,YES
2,propn,8.5,25.072703,0.00000,0.001274,YES
3,adv,8.5,51.636507,0.00000,0.001274,YES
4,intj,26.5,15.712987,0.00039,0.001653,YES
5,cconj,82.0,0.462533,0.79353,0.020000,NO
6,sconj,36.0,12.410101,0.00202,0.001961,NO
7,adp,74.0,0.663410,0.71770,0.007692,NO
8,det,39.0,11.807713,0.00273,0.002083,NO
9,num,58.0,2.656925,0.26488,0.003448,NO


['noun', 'adj', 'propn', 'adv', 'intj', 'punct', 'pron', 'nominals', 'TTR', 'avg_word_len', 'pron/noun_ratio', 'see_pron', '1st_pron_sg', '2nd_pron_sg', '1st_pron_pl', '3rd_pron_pl', 'active_voice', 'passive_voice', '1st_prs_verb', '2nd_prs_verb', 'core_verb', 'finite_verb', 'supine', 'discourse', 'ind_mood', 'imp_mood', 'neg_polarity', 'gen_case', 'ade_case', 'nsubj_cop', 'modal', 'csubj_cop', 'obl', 'nmod', 'cop']

NOUN
              1         2             3
1  1.000000e+00  0.032324  2.339419e-09
2  3.232403e-02  1.000000  2.478731e-02
3  2.339419e-09  0.024787  1.000000e+00
Kui p>0.05, siis need jäetakse välja!

GRUPP 1 (tugev/mõõdukas vs nõrk)
noun, 0.0323240309 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.224; 0.217
NÕRK -> mediaan/keskmine: 0.263; 0.263
MITTEEKSISTEERIV -> mediaan/keskmine: 0.306; 0.311
GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
noun, 2.3e-09 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.224; 0.217
NÕRK -> mediaan/keskmine: 0.263; 0.263
MITTEEKSISTEERIV -> mediaan/keskm


1ST_PRON_PL
          1         2         3
1  1.000000  0.973176  0.000008
2  0.973176  1.000000  0.000413
3  0.000008  0.000413  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
1st_pron_pl, 7.9024e-06 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.003; 0.005
NÕRK -> mediaan/keskmine: 0.004; 0.005
MITTEEKSISTEERIV -> mediaan/keskmine: 0.0; 0.0
GRUPP 3 (nõrk vs MITTEEKSISTEERIV)
1st_pron_pl, 0.000412971 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.003; 0.005
NÕRK -> mediaan/keskmine: 0.004; 0.005
MITTEEKSISTEERIV -> mediaan/keskmine: 0.0; 0.0

3RD_PRON_PL
          1         2         3
1  1.000000  0.292783  0.000510
2  0.292783  1.000000  0.141806
3  0.000510  0.141806  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
3rd_pron_pl, 0.0005100155 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.0; 0.004
NÕRK -> mediaan/keskmine: 0.0; 0.002
MITTEEKSISTEERIV -> mediaan/keskmine: 0.0; 0.001

ACTIVE_VOICE
          1      


CSUBJ_COP
          1         2         3
1  1.000000  0.797835  0.001518
2  0.797835  1.000000  0.029047
3  0.001518  0.029047  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
csubj_cop, 0.0015176267 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.0; 0.002
NÕRK -> mediaan/keskmine: 0.0; 0.002
MITTEEKSISTEERIV -> mediaan/keskmine: 0.0; 0.0
GRUPP 3 (nõrk vs MITTEEKSISTEERIV)
csubj_cop, 0.0290467739 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.0; 0.002
NÕRK -> mediaan/keskmine: 0.0; 0.002
MITTEEKSISTEERIV -> mediaan/keskmine: 0.0; 0.0

OBL
          1         2         3
1  1.000000  0.272493  0.000164
2  0.272493  1.000000  0.097205
3  0.000164  0.097205  1.000000
Kui p>0.05, siis need jäetakse välja!

GRUPP 2 (tugev/mõõdukas vs MITTEEKSISTEERIV)
obl, 0.0001641515 

TUGEV/MÕÕDUKAS -> mediaan/keskmine: 0.083; 0.084
NÕRK -> mediaan/keskmine: 0.09; 0.094
MITTEEKSISTEERIV -> mediaan/keskmine: 0.11; 0.114

NMOD
              1         2             3
1  1.000

## KORRELATSIOONID DIMENSIOONIDEGA

In [23]:
# dimensioonide vahel spearman!

f = 'limesurvey_loplikud_skoorid_koond.csv'
# f = 'limesurvey_loplikud_skoorid_ordinal.csv'

data = []
with open(f, 'r') as fid:
    csv_reader = csv.DictReader(fid, delimiter=',')
    fieldnames = csv_reader.fieldnames
    for line in csv_reader:
        data.append(line)

col_list1 = ['abs_avg1', 'info_avg1', 'aeg_avg1', 'afek_avg1', 'inter_avg1', 'inst_avg1', 'form_avg1', 'keer_avg1', 'subj_avg1', 'spont_avg1', 'imp_avg1', 'arg_avg1']
col_list2 = ['abs_avg2', 'info_avg2', 'aeg_avg2', 'afek_avg2', 'inter_avg2', 'inst_avg2', 'form_avg2', 'keer_avg2', 'subj_avg2', 'spont_avg2', 'imp_avg2', 'arg_avg2']

# mis on vahet nt abs_avg1 ja abs_avg2 vahel?  et oli kaks gruppi hindajaid. nt ühelt grupilt tuleb keskmiseks 1.5, teiselt grupilt keskmiseks 1.4, siis
# keskmine tuleb 1.45. kui aga juhtub, et ühelt grupilt saab 1 ja teise grupi hinnangut ei tea üldse (st saab ##)
# siis DIM_avg1 korral tehakse (1 + ##)2 = 1
# ja DIM_avg2 korral saab automaatselt -1 (ehk ei tea midagi selle teksti kohta)
# ja kui mõlemad grupid annavad ## ja ##, siis mõlemad saavad keskmiseks ka -1
# valituks osutus: avg1 veerg

dim_df = pd.DataFrame(data).iloc[:, 1:].astype(float)
dim_ver1 = dim_df[col_list1]
dim_ver1




,abs_avg1,info_avg1,aeg_avg1,afek_avg1,inter_avg1,inst_avg1,form_avg1,keer_avg1,subj_avg1,spont_avg1,imp_avg1,arg_avg1
0,1.450000,1.666667,0.250000,0.800000,0.533333,1.250000,0.4,0.500000,1.300000,0.250,2.000,2.00
1,1.000000,2.000000,0.250000,1.375000,1.000000,1.041667,-1.0,0.600000,2.000000,0.125,0.500,1.00
2,1.500000,-1.000000,-1.000000,2.200000,1.166667,0.750000,0.0,-1.000000,2.333333,0.700,0.670,1.50
3,0.800000,-1.000000,1.000000,1.600000,0.300000,0.300000,0.4,0.000000,2.200000,0.225,1.400,2.00
4,1.250000,1.333333,-1.000000,1.000000,0.200000,0.250000,0.4,0.600000,1.050000,0.500,0.670,1.45
...,...,...,...,...,...,...,...,...,...,...,...,...
115,0.433333,-1.000000,2.100000,2.200000,1.700000,0.666667,0.2,0.166667,1.400000,-1.000,1.025,0.80
116,0.200000,1.333333,0.733333,1.916667,0.750000,0.000000,0.5,0.200000,2.100000,0.600,1.500,1.80
117,0.000000,2.875000,2.600000,1.083333,0.791667,1.000000,1.0,1.000000,0.250000,0.200,1.250,-1.00
118,0.500000,2.125000,-1.000000,0.200000,0.500000,2.875000,0.2,0.250000,-1.000000,-1.000,1.600,-1.00


In [24]:
def remove_elements(ls1: list[int], ls2: list[int], equal_to: int = -1) -> tuple[list[int], list[int]]:
    indices = {i for i, el in enumerate(ls1) if el == equal_to} | {i for i, el in enumerate(ls2) if el == equal_to}
    return [el for i, el in enumerate(ls1) if i not in indices], [el for i, el in enumerate(ls2) if i not in indices]

In [25]:
dim_pairs = [pair for pair in itertools.combinations(col_list1,2)]
for p in dim_pairs:
#     print(len(dim1_arr), len(dim2_arr))
    arr1, arr2 = remove_elements(dim_df[p[0]], dim_df[p[1]])
#     print(len(arr1), len(arr2))
    
    sp = stats.spearmanr(arr1, arr2)
    correlation = sp.correlation
    pval = sp.pvalue
#     if pval >= 0.05:
#         print(p)
#         print(round(pval, 5))

## KORRELATSIOONID TUNNUSTEGA

In [26]:
input_f = 'limesurvey_results4.csv'

data = []

with open(input_f, 'r') as fid:
    csv_reader = csv.DictReader(fid, delimiter=',')
    fieldnames = csv_reader.fieldnames
    for line in csv_reader:
        data.append(line)

feature_df = pd.DataFrame(data).iloc[:, 1:].astype(float)
feature_df

FileNotFoundError: [Errno 2] No such file or directory: 'limesurvey_results4.csv'

In [ ]:
feature_names = feature_df.columns
feature_names

In [ ]:
def save_csv(prefix, df):
    df.to_csv(f'korrelatsiooni_csvd/{prefix}.csv')

In [ ]:
def plot_important_features(model, dimname):
    
    print(model.coef_)

    feature_importance = abs(model.coef_[0])
    
    
    feature_importance = 1.0 * (feature_importance / feature_importance.max())
    sorted_idx = np.argsort(feature_importance)
    pos = np.arange(sorted_idx.shape[0]) + .5

    featfig = plt.figure(figsize=(15,20))
    featax = featfig.add_subplot(1, 1, 1)

    featax.barh(pos, feature_importance[sorted_idx], align='center')
    featax.set_yticks(pos)
    featax.set_yticklabels(np.array(X.columns)[sorted_idx], fontsize=10)
    featax.set_xlabel(f'Relative Feature Importance {str.upper(dimname)}')

    # plt.tight_layout()   
    plt.show()

In [ ]:
def get_corr_pairs(corr_df):

    columns = [line for line in corr_df]
    matches = []

    for column, rows in corr_df.items():
        for m, score in rows.items():
            if (score > 0.6 or score < -0.6) and column != m:
                matches.append([(column, m), score])
            continue

    without_duplicates = []
    for ws, i in matches:
        ls = (sorted(ws), i)
        if ls not in without_duplicates:
            without_duplicates.append(ls)       

    return without_duplicates

In [ ]:
def generate_heatmap(dim, corr_data):

    my_colors = ['black', 'lightgrey', 'white', 'red']
    my_cmap = ListedColormap(my_colors)
    bounds = [-1.0, -0.6, 0.6, 1.0]
    my_norm = BoundaryNorm(bounds, ncolors=len(my_colors))



    mask = np.triu(np.ones_like(corr_data))

    fig, ax = plt.subplots(1, 1, figsize=(20,20))
    hmap = sns.heatmap(corr_data,
                yticklabels=1, 
                ax=ax,
                linewidths=1.0,
                cmap=my_cmap,
                norm=my_norm,
                mask=mask,
               cbar_kws = dict(use_gridspec=False,location="top")
               )

    colorbar = ax.collections[0].colorbar
    hmap.figure.savefig(f'heatmapid/{dim}_heatmap.png', format='png', dpi=150)
    plt.show()


In [ ]:

all_corr = feature_df.corr(method='spearman').round(2)

# save_csv('dimensions', all_corr)

In [ ]:
generate_heatmap('all_dims', all_corr)

In [ ]:
pairs = get_corr_pairs(all_corr)
for pair in pairs:
    print(f'{pair[0]} = {pair[1]}')

# # Lineaarsuse eeldus

Pearsoni jaoks

In [ ]:
feature_pairs = [pair for pair in itertools.combinations(feature_names,2)]

In [ ]:
# with PdfPages('foo.pdf') as pdf:
#     for x, y in feature_pairs:

#         figure = plt.figure()
#         ax = plt.gca()
#         ax.scatter(feature_df[x], feature_df[y])
#         ax.set_xlabel(x)
#         ax.set_ylabel(y)
#     #     ax.set_title("{} vs {}".format(x, y))
#         plt.show()
#         pdf.savefig(figure)
    